In [226]:
# A 1-cylinder PTS corresponds to a tuple of four elements (a,b,c,d) in Sym(n)
# such that abcd = 1. Our goal is to look for non-cyclic PTS, i.e. tuples such 
# that <a,b,c,d> is a non-cyclic subgroup of Sym(n).
from sage.all import SymmetricGroup
# a constellation is a tuple of permutations whose product is the identity. 
# the length of a constellation is the number of elements in the tuple,
# and the degree is the n of the symmetric group the permutations come from.
# this library by default only returns connected constellations, i.e. 
# constellations whose permutations have a connected orbit when acting on 
# {1,...,n} in order to be 1-cyl, a surface must be represented by a connected 
# constellation, so this is ok.
from sage.combinat import constellation

n = 5
Sn = SymmetricGroup(n)
all_pts = Constellations(4,n)
# tuple.g(i) has type sage.groups.perm_gps.permgroup_element.SymmetricGroupElement'
horizontal_one_cyl_set = set()
for tuple in all_pts:
    if (tuple.g(0)*tuple.g(1)).cycle_type() == [n]:
        horizontal_one_cyl_set.add(tuple)
print(f'Total number of degree {n} PTS: {len(all_pts)}')
print(f'Total number of degree {n} PTS with a horizontal cyl: {len(horizontal_one_cyl_set)}')

Total number of degree 5 PTS: 1647384
Total number of degree 5 PTS with a horizontal cyl: 345600


In [238]:
def check_horizontal_direction(tuple,n):
    # print(f'Checking if {tuple.g(0)}*{tuple.g(1)}={tuple.g(0)*tuple.g(1)} is an n-cycle')
    if (tuple.g(0)*tuple.g(1)).cycle_type() == [n]:
        return True 
    else:
        return False
    
def check_one_cyl_surface(tuple,n):
    G = tuple.braid_group_orbit()
    for t in G.vertices(): # t not nec = tuple, but will be iso as consts 
    # can actually get the original perm here if we want, using t.is_isomorphic(tuple) (returns r) and then conjugating t.relabel(r) = tuple
        if not check_horizontal_direction(t,n):
            return False
    return True

def check_for_identity(tuple,n):
    for i in range(0,4):
        if tuple.g(i).cycle_type() == ([1]*n):
            return True 
    return False


def display_info(tuple,n):
    print(f'Information about {tuple}\n')
    if check_one_cyl_surface(tuple,n):
        print(f'one-cylinder: yes\n')
    else:
        print(f'one-cylinder: no\n')
    
    print(f'Tuple profile: {tuple.profile()}\n')

    print(f'B4 orbit:')
    for t in tuple.braid_group_orbit().vertices():
        print(f'{t}')
        print(f'a\'b\' = {t.g(0)*t.g(1)}')

In [230]:
test_set = horizontal_one_cyl_set.copy()
count = 0
interesting = set() # orbit rep of one cylinder surfaces, i.e. a'b' is an n-cycle
skipped = 0

for tuple in horizontal_one_cyl_set:
    if check_one_cyl_surface(tuple,n):
        interesting.add(tuple)
    else:
        skipped += 1
    # try:
    #     test_set.remove(tuple)
    # except:
    #     skipped +=1 
    #     continue
    # G = tuple.braid_group_orbit()
    # iso_class_already_found = False
    # for t in G.vertices(): # this is working with isomorphism classes 
    #     try:
    #         test_set.remove(t)
    #     except: # means we've already removed the iso class from the set
    #         skipped+=1
    #         iso_class_already_found = True
    # if not iso_class_already_found:
    #     for t in G.vertices():
    #         if not check_horizontal_direction(t,n):
    #             break
    #     interesting.add(G.vertices()[0])
        
    
print(len(interesting))
print(skipped) # suspicious that this is 0, maybe has to do with iso?
print(len(interesting)+skipped)

69120
276480
345600


In [231]:

print(f'Total number of degree {n} one-cylinder PTS: {len(interesting)}')

Total number of degree 5 one-cylinder PTS: 69120


In [232]:
cyclic_one_cyl = set()
non_cyclic_one_cyl = set()
for tuple in interesting:
    G = Sn.subgroup([tuple.g(0), tuple.g(1), tuple.g(2), tuple.g(3)])
    cyclic_one_cyl.add(tuple) if G.is_cyclic() else non_cyclic_one_cyl.add(tuple)

print(f'Number of degree {n} one-cylinder PTS that are: \n cyclic: {len(cyclic_one_cyl)}')

non_cyclic_one_cylinder_non_identity = set()
identity = set()
for tuple in non_cyclic_one_cyl:
    if check_for_identity(tuple,n):
        identity.add(tuple) 
    else:
        non_cyclic_one_cylinder_non_identity.add(tuple)
        
print(f' non-cyclic with identity perm: {len(identity)}')
print(f' non-cyclic without identity perm: {len(non_cyclic_one_cylinder_non_identity)}\n')


Number of degree 5 one-cylinder PTS that are: 
 cyclic: 480
 non-cyclic with identity perm: 1920
 non-cyclic without identity perm: 66720



In [236]:
cycle_types_set = set()
for tuple in non_cyclic_one_cylinder_non_identity:
    # for perm in SymmetricGroup(4):
    #     if perm.action(list(tuple)) in cycle_types_set:
    #         print('foo')
        cycle_types_set.add(tuple.profile())

print(f'Number of cycle types: {len(cycle_types_set)}')

dedup_cycle_types_set = set()
for t in cycle_types_set:
    duplicate = False
    for perm in SymmetricGroup(4):
        permuted = (t[perm(1)-1], t[perm(2)-1], t[perm(3)-1], t[perm(4)-1])
        if permuted in dedup_cycle_types_set:
            duplicate = True
        
    if not duplicate:
         dedup_cycle_types_set.add(t)
print(f'Number of cycle types up to permutation: {len(dedup_cycle_types_set)}') # note: using iso type above produces same list
for t in dedup_cycle_types_set:
     print(t)
    

Number of cycle types: 136
Number of cycle types up to permutation: 26
([2, 2, 1], [2, 2, 1], [5], [2, 2, 1])
([4, 1], [2, 1, 1, 1], [2, 1, 1, 1], [4, 1])
([3, 2], [3, 2], [4, 1], [4, 1])
([4, 1], [2, 1, 1, 1], [4, 1], [4, 1])
([3, 1, 1], [3, 1, 1], [5], [5])
([5], [5], [5], [3, 1, 1])
([2, 2, 1], [3, 1, 1], [2, 2, 1], [5])
([4, 1], [2, 1, 1, 1], [3, 2], [3, 2])
([5], [5], [5], [2, 2, 1])
([4, 1], [4, 1], [4, 1], [3, 2])
([3, 2], [4, 1], [4, 1], [2, 1, 1, 1])
([3, 1, 1], [3, 1, 1], [5], [3, 1, 1])
([2, 2, 1], [3, 1, 1], [3, 1, 1], [3, 1, 1])
([2, 2, 1], [5], [2, 2, 1], [5])
([5], [3, 1, 1], [2, 2, 1], [5])
([3, 1, 1], [2, 2, 1], [3, 1, 1], [2, 2, 1])
([4, 1], [2, 1, 1, 1], [2, 1, 1, 1], [3, 2])
([2, 2, 1], [2, 2, 1], [2, 2, 1], [2, 2, 1])
([3, 2], [4, 1], [3, 2], [3, 2])
([2, 2, 1], [2, 2, 1], [3, 1, 1], [2, 2, 1])
([4, 1], [4, 1], [4, 1], [4, 1])
([3, 1, 1], [3, 1, 1], [3, 1, 1], [3, 1, 1])
([3, 1, 1], [2, 2, 1], [5], [3, 1, 1])
([5], [5], [5], [5])
([3, 2], [2, 1, 1, 1], [3, 2], [3, 

In [239]:
for tuple in non_cyclic_one_cylinder_non_identity:
    display_info(tuple,n)
    break

Information about Constellation of length 4 and degree 5
g0 (1,4,3,2)(5)
g1 (1)(2)(3,5)(4)
g2 (1,3,5,2)(4)
g3 (1,3,2,4)(5)

one-cylinder: yes

Tuple profile: ([4, 1], [2, 1, 1, 1], [4, 1], [4, 1])

B4 orbit:
Constellation of length 4 and degree 5
g0 (1)(2,3,4,5)
g1 (1,2)(3)(4)(5)
g2 (1,3,4,2)(5)
g3 (1)(2,3,5,4)
a'b' = (1,2,3,4,5)
Constellation of length 4 and degree 5
g0 (1)(2)(3)(4,5)
g1 (1,2,3,4)(5)
g2 (1,2,5,4)(3)
g3 (1,3,2,5)(4)
a'b' = (1,2,3,4,5)
Constellation of length 4 and degree 5
g0 (1)(2,3,4,5)
g1 (1,2,3,5)(4)
g2 (1,2)(3)(4)(5)
g3 (1)(2,4,3,5)
a'b' = (1,2,5,3,4)
Constellation of length 4 and degree 5
g0 (1)(2,3,4,5)
g1 (1,2)(3)(4)(5)
g2 (1)(2,3,5,4)
g3 (1,5,2,3)(4)
a'b' = (1,2,3,4,5)
Constellation of length 4 and degree 5
g0 (1)(2,3,4,5)
g1 (1,2)(3)(4)(5)
g2 (1,4,5,2)(3)
g3 (1)(2,4,5,3)
a'b' = (1,2,3,4,5)
